In [251]:
path="/Users/pragnyasuresh/Desktop/team_no_8/"

In [249]:
import spacy
import textacy
from textacy import lexicon_methods
from nltk.corpus import wordnet as wn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import os
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import wordnet as wn
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords 
import math
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.utils import to_categorical
import numpy as np
from numpy import array 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
from spacy.parts_of_speech import ADJ, ADV, NOUN, VERB
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D
from keras.models import Sequential
from keras.layers import Input, Flatten
from keras.layers import Dense, Activation
from keras.layers import MaxPooling1D
from os import path
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import scipy.sparse as sps
from spacy.lang.en import English
from ahocorasick import Automaton
from spacy.util import minibatch, compounding
from IPython.display import Markdown, display

In [ ]:
'''train a model  to categorize the reviews based on positive and negative sentiments. So output of svm will be 1  and 1
0-negative
Take the negative sentiment reviews and apply ner
These are the reviews that have issues
Once the entities have been recognised, the review can be routed to the corresponding dept

Ex:  Review: few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead. 
     Entities : (PRODUCT,top), (INFO,cotton),(INFO,nylon)
     Response :What was the brand of the top?
'''

In [ ]:
#Categories:
#1. Clothing
#2. Phones
#3. Home
#4. Footwear

In [253]:
path="/Users/pragnyasuresh/Desktop/team_no_8/"

In [255]:
#Extracting 4000 reviews from each category and performing preprocessing-stop word removal and lemmatization 
stop_words = set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
with open(path + "Clothing_Accessories.txt", 'r') as f:
    with open(path + "reviews.csv", 'w') as writeFile:
        original_reviews=[]
        writer = csv.writer(writeFile)
        header=["department","sentiment","review","original_review"]
        writer.writerow(header)
        ctr=0
        for line in f:
            if(line.find("review/score:")!=-1):
                rev=["Clothing_Accessories",line[13:]]
            if(line.find("review/text:")!=-1 and ctr<4000):
                review=str(line[13:]).lower()
                #print(review)
                lll = [lemmatizer.lemmatize(w).lower() for w in review.split(' ') if w.lower() not in stop_words]
                rev.append(' '.join(lll))
                original_reviews.append(review)
                #print(rev)
                rev.append(review)
                writer = csv.writer(writeFile)
                writer.writerow(rev)
                ctr+=1
    print(ctr)

4000


In [256]:
stop_words = set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
with open(path+"Cell_Phones_Accessories.txt", 'r') as f:
    with open(path+"reviews.csv", 'a') as writeFile:
        ctr=0
        for line in f:
            if(line.find("review/score:")!=-1):
                rev=["Cell_Phones_Accessories",line[13:]]
            if(line.find("review/text:")!=-1 and ctr<4000):    
                review=str(line[13:]).lower()
                #print(review)
                lll = [lemmatizer.lemmatize(w).lower() for w in review.split(' ') if w.lower() not in stop_words]
                rev.append(' '.join(lll))
                original_reviews.append(review)
                rev.append(review)
                writer = csv.writer(writeFile)
                writer.writerow(rev)
                ctr+=1
            
            
    print(ctr)

4000


In [257]:
stop_words = set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
with open(path+"Home_Kitchen.txt", 'r') as f:
    with open(path+"reviews.csv", 'a') as writeFile:
        ctr=0
        for line in f:
            if(line.find("review/score:")!=-1):
                    rev=["Home_Kitchen",line[13:]]
            if(line.find("review/text:")!=-1 and ctr<4000):
                review=str(line[13:]).lower()
                lll = [lemmatizer.lemmatize(w).lower() for w in review.split(' ') if w.lower() not in stop_words]
                rev.append(' '.join(lll))
                original_reviews.append(review)
                rev.append(review)
                writer = csv.writer(writeFile)
                
                writer.writerow(rev)
                ctr+=1
            
            
    print(ctr)

4000


In [258]:
#Drop all the reviews with rating 3. Update reviews with rating 1 & 2 as 0 rating. Update Reviews with
#4 and 5 as 1 rating. We are doing this to make it a binary classification.
df=pd.read_csv(path+"reviews.csv", delimiter = ',')
print(len(df))
print(df['sentiment'].value_counts())

#Drop all the reviews with rating 3
df = df[df.sentiment != 3]
print(len(df))
df['sentiment']=df['sentiment'].replace([1, 2], 0)
df['sentiment']=df['sentiment'].replace([4, 5], 1)
print(df['sentiment'].value_counts())


12000
5.0    6064
4.0    2349
1.0    1530
3.0    1120
2.0     937
Name: sentiment, dtype: int64
10880
1.0    8413
0.0    2467
Name: sentiment, dtype: int64


In [259]:
#Making the dataset balanced by dropping reviews of the class that is more in number.
df1=df.loc[df['sentiment'] == 1]
df2=df.loc[df['sentiment'] == 0]

df1=df1[:3773]
print(len(df1),len(df2))

df=df1.append(df2, ignore_index=True)
print(df['department'].value_counts())

3773 2467
Clothing_Accessories       3650
Cell_Phones_Accessories    2021
Home_Kitchen                569
Name: department, dtype: int64


In [260]:
#Creating sparse vector using TFID vectorizer
vectorizer = TfidfVectorizer()
sparse_vector = vectorizer.fit_transform(df["review"])
n_features=sparse_vector.shape[1]

In [261]:
#Using SVM to classify reviews as positive or negative.
#Since we needed to manually annotate the corpus for training the ner, we could not use a huge corpus
#ANN needs huge amount of data to provide good accuracy.
#SVM classification gave much better accuracy whereas ANN gave about 72% so we decided to use SVM to classify the reviews
X_train, X_test, y_train, y_test = train_test_split(sparse_vector, df["sentiment"], test_size=0.4, random_state=42)
l1=list(df["department"])
l2=list(df["review"])
l3=list(df["original_review"])
l=[]
for i in range(len(l1)):
    l.append([l1[i],l2[i],l3[i]])

#need to map the reviews and department to the review. To do this, the line below is used. It will split the same way
X_train2, X_test2, y_train2, y_test2 = train_test_split(l, df["sentiment"], test_size=0.4, random_state=42)
clf = SVC(kernel='linear') 
# fitting x samples and y classes 
clf.fit(X_train, y_train) 
#Predicting and evaluatng accuracy

y_pred_list=clf.predict(X_test)
ans=array(y_pred_list)
acc = sklearn.metrics.accuracy_score(y_test, y_pred_list)
print("Accuracy using sparse vector representation of each review =",acc*100)


Accuracy using sparse vector representation of each review = 92.2275641025641


In [262]:
#we now extract all reviews with a negative sentiment to perform NER on them
y_pred_list=list(y_pred_list)
df_pred = pd.DataFrame({'sent':y_pred_list})

print (df_pred["sent"].value_counts())
X_test2=list(X_test2)
negative_reviews=[]
j=0
with open(path+"negative_reviews.csv", 'w') as writeFile:
    header=["department","review","original_review"]
    writer = csv.writer(writeFile)
    writer.writerow(header)
    for i in y_pred_list:
    
        if(i==0.0):
            #print(i,j)
            negative_reviews=[X_test2[j][0]]
            negative_reviews.append(X_test2[j][1])
            negative_reviews.append(X_test2[j][2])
            #print(negative_reviews)
            writer = csv.writer(writeFile)
            writer.writerow(negative_reviews)
        j+=1
df_neg = pd.read_csv(path+"negative_reviews.csv") 
df_neg["department"].value_counts()

1.0    1567
0.0     929
Name: sent, dtype: int64


Cell_Phones_Accessories    458
Clothing_Accessories       246
Home_Kitchen               225
Name: department, dtype: int64

In [263]:
#Creating the train and test datasets for the NER model (750-train, 258-test)
df_neg['department']=df_neg['department'].replace("Clothing_Accessories", 1)
df_neg['department']=df_neg['department'].replace("Cell_Phones_Accessories", 2)
df_neg['department']=df_neg['department'].replace("Home_Kitchen", 3)
train= df_neg.iloc[:750, :]
test = df_neg.iloc[750:,:]
test=df_neg.drop(["department"],axis=1)
train.to_csv(path+"train_ner.csv",index=False)
test.to_csv(path+"test_ner.csv",index=False)


In [ ]:
#perform ner on negative_reviews - training the NER model
from __future__ import unicode_literals, print_function
import patterns
import plac
import random
from pathlib import Path
import spacy
import csv
import pandas as pd

df_ent=pd.read_csv(path+"train_ner.csv")
e=df_ent["review"]
e=e.values.tolist()
TRAIN_DATA = []
patterns.A.make_automaton()

for x in e:
    list_ent=[]
    d={}
    for item in patterns.A.iter(x):
        val=((item[0]-len(item[1][1]))+1,item[0]+1,item[1][0])
        list_ent.append(val)
    d["entities"]=list_ent
    TRAIN_DATA.append((x,d))


df_test=pd.read_csv(path+"test_ner.csv")
test=df_test["review"]
TEST_DATA=list(test)


@plac.annotations(
                  model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
                  output_dir=("/Users/pragnyasuresh/Desktop/team_no_8", "option", "o", Path),
                  n_iter=("Number of training iterations", "option", "n", int),
                  )
def main(model=None, output_dir=path, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            #print(ent)
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
            for itn in range(n_iter):
                random.shuffle(TRAIN_DATA)
                losses = {}
                # batch up the examples using spaCy's minibatch
                batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
                for batch in batches:
                    texts, annotations = zip(*batch)
                    nlp.update(
                               texts,  # batch of texts
                               annotations,  # batch of annotations
                               drop=0.5,  # dropout - make it harder to memorise data
                               losses=losses,
                               )
                Lprint("Losses", losses)


    # test the trained model
    l=[]
    l2=[]
    for text  in TEST_DATA:
        doc = nlp(text)
        #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        for ent in doc.ents:
            l.append(ent.text)
            l2.append(ent.label_)

        df = pd.DataFrame({'ent':l,'label':l2})
        df.to_csv(path+"test.csv",index=False)
#print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        
        # save model to output directoryf
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
        
        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == "__main__":
    plac.call(main)

'''
    it’s not enough to only show a model a single example once. Especially if you only have few examples, you’ll want to train for a number of iterations. At each iteration, the training data is shuffled to ensure the model doesn’t make any generalizations based on the order of examples. Another technique to improve the learning results is to set a dropout rate, a rate at which to randomly “drop” individual features and representations. This makes it harder for the model to memorize the training data. For example, a 0.25 dropout means that each feature or internal representation has a 1/4 likelihood of being dropped.'''




In [ ]:
#Testing the NER model after training
from __future__ import unicode_literals, print_function
import patterns
import plac
import random
from pathlib import Path
import csv
import pandas as pd
from spacy.util import minibatch, compounding


df_test=pd.read_csv(path+"test_ner.csv")
test=df_test["review"]
TEST_DATA=list(test)

@plac.annotations(
                  model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
                  output_dir=("/Users/pragnyasuresh/Desktop/team_no_8", "option", "o", Path),
                  n_iter=("Number of training iterations", "option", "n", int),
                  )
def main(model=None, output_dir="/Users/pragnyasuresh/Desktop/team_no_8", n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    # test the trained model
    l=[]
    l2=[]

    with open('/Users/pragnyasuresh/Desktop/team_no_8/test.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        header=["Entity","Label","Review"]
        writer.writerow(header)
        for text  in TEST_DATA:
            doc = nlp2(text)
            l3=text
            #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            l1=[]
            l2=[]
            for ent in doc.ents:
                if(ent.text not in l1):
                    l1.append(ent.text)
                    l2.append(ent.label_)
        
            l=[','.join(l1),','.join(l2),l3]
            print(l)
            writer.writerow(l)


if __name__ == "__main__":
    plac.call(main)



In [265]:
Departments = ["Clothing","Footwear","Home","Phones"]
Clothing = ["shirt","tee","jeans","pant","tshirt","t-shirt","top","shorts","saree","fabric","skirt","joggers","dress"]
Footwear = ["shoe","slipper","heels","bellies","sandals","boots","shoes"]
Home = ["pan","cooker","warmer","coffee","mug","cup","china","lunchbox","fan","mixer","juicer","curtain","cookware"]
Phones = ["phone","charger","adapter","plug","cable","cord","earset","ear","buds","display","case","battery","headset","earphones","bluetooth","cover"]
Ent=Clothing+Footwear+Home+Phones
Department = [Clothing,Footwear,Home,Phones]

In [285]:
#Obtaining synset meanings for the words that describe each department and providing them as input to train Naive Bayes
#Classifier
stop_words = set(stopwords.words('english'))
dfdept = []
for department in range(len(Departments)):
    w1=wn.synsets(Departments[department])[0]
    dept = []
    for ele in Department[department]:
        w2=wn.synsets(ele)
        #w1.wup_similarity()
        if(len(w2)>0):
            maxsim = 0
            maxindex = -1
            for i in range(len(w2)):
                #print(w1,w2[i],i)
                if(w2[i].name().split('.')[1] == 'n'):
                    sim = w1.wup_similarity(w2[i])
                    #print(sim)
                    if(type(sim) is float and sim > maxsim):
                        maxsim = sim
                        maxindex = i
                else:
                    break
            
            string = ele + " " + w2[maxindex].definition()
            #print("string:",string.split(' '))
            lll = [w.lower() for w in string.split(' ') if w.lower() not in stop_words]
            dept.append(' '.join(lll))
            #dept.append(string)
            #print(dept)
            
        else:
            dept.append(ele)
            
    dfdept.append(dept)
#print("dfdept:",dfdept[0])
#df.{"meaning":"a garment" , "department":0k}

df_train = pd.DataFrame({'meaning':dfdept[0], 'department': 0})
#df_train = df.append(pd.DataFrame({'meaning':dfdept[1], 'department': 1}))

for i in range(1,len(dfdept)):
    df2 = pd.DataFrame({'meaning':dfdept[i], 'department': i})
    df_train = df_train.append(df2)
#df_train=df_train.sample(frac=1)
print(df_train)


    department                                            meaning
0            0                 shirt garment worn upper half body
1            0  tee support holding football end ground prepar...
2            0  jeans (usually plural) close-fitting trousers ...
3            0  pant (usually plural) garment extending waist ...
4            0                                             tshirt
5            0               t-shirt close-fitting pullover shirt
6            0  top garment (especially women) extends shoulde...
7            0                           shorts trousers end knee
8            0  saree dress worn primarily hindu women; consis...
9            0  fabric artifact made weaving felting knitting ...
10           0  skirt garment hanging waist; worn mainly girls...
11           0  joggers someone runs steady slow pace (usually...
12           0  dress clothing distinctive style particular oc...
0            1  shoe footwear shaped fit foot (below ankle) fl...
1         

In [267]:
#Routing the reviews to the corresponding Department using Naive Bayes
#CountVectorizer is used to count the number of times a word occurs in a corpus
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df_train['meaning'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df_train['department'])
predicted=[]
docs_new=pd.read_csv(path+"test.csv")
docs_new1=docs_new["Entity"]
docs_new2=docs_new["Label"]
docs_new3=docs_new["Original_review"]
docs_new=pd.concat([docs_new1,docs_new2,docs_new3],axis=1)
docs_new = docs_new.dropna()
for i in docs_new["Entity"]:
    #print("i:",i)
    l=[]
    x=i.split(',')
    for ele in x:
        w=wn.synsets(ele)
    
        if(len(w)==0):
            l=[i]    
        else:
            string=ele+" "+w[0].definition()
            s=[w.lower() for w in string.split(' ') if w.lower() not in stop_words]
            l.append(' '.join(s))
    l=','.join(l)
    X_new_counts = count_vect.transform([l])
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    predicted.append(clf.predict(X_new_tfidf))

with open(path+"named_entities.csv", 'w') as writeFile:
    writer = csv.writer(writeFile)
    header=["Entities","Label","Department","Original_review"]
    writer.writerow(header)
    
    for doc, label,category,rev in zip(docs_new["Entity"],docs_new["Label"], predicted,docs_new["Original_review"]):
        named_entities=[]
        print('%r => %s' % (doc,Departments[category[0]]))
        named_entities=[doc,label,Departments[category[0]],rev]
        writer = csv.writer(writeFile)
        writer.writerow(named_entities)
    



'phone,case,cover,belt,wear' => Phones
'top' => Clothing
'phone,battery,motorola,samsung,compared' => Phones
'belt,cover,small' => Phones
'shirt' => Clothing
'dockers,coffee' => Home
'under armour,year' => Phones
'shirt,dress,ordered,red' => Clothing
'motorola,phone,nokia' => Phones
'charger' => Phones
'phone,case' => Phones
'phone,bluetooth,battery' => Phones
'phone,battery' => Phones
'phone' => Phones
'phone,~3,year,verizon,battery,charger' => Phones
'case' => Phones
'cookware,pan' => Home
'display' => Phones
'phone' => Phones
'display,large,red,stored,warmer' => Home
'year' => Phones
'wear,shirt,dress,ordered' => Clothing
'phone' => Phones
'phone' => Phones
'year' => Phones
'ordered,charger,nokia,phone' => Phones
'phone,black,sony,cable,charger' => Phones
'charger' => Phones
'coffee,warmer,year,young,mug,merry,black' => Home
'coffee,mug' => Home
'top' => Clothing
'samsung,battery,year' => Phones
'coffee,black,baby.i' => Home
'pant,maroon,ordered,size,year,wear' => Phones
'cooker,sma

In [268]:
#Generating an automated response based on the entities recognised
df=pd.read_csv(path+"named_entities.csv", delimiter = ',')
#print(x['Entities'])
#print(x['label'])

with open(path+"response.csv",'w') as writeFile:
    header=["Response"]
    writer = csv.writer(writeFile)
    writer.writerow(header)
    for x in range(len(df)):
        #print(df.iloc[x])
        brand=0
        product=0
        info=0
        cloth=0
        size=0
        gender=0
        b=[]
        p=[]
        inf=[]
        #print(x,df.iloc[x]['Label'])
        labels=df.iloc[x]['Label'].split(',')
        #print(labels)
        entities=df.iloc[x]['Entities'].split(',')
        for j in range(len(labels)):
            #print(j,labels[j])
            if labels[j]=="BRAND":
                brand=1
                b.append(entities[j])
            elif labels[j]=="PRODUCT":
                product=1
                p.append(entities[j])
            #if df.iloc[x]['Department']=="Phones":
            elif labels[j]=="INFO":
                info=1
                inf.append(entities[j])
            if df.iloc[x]['Department']=="Clothing":
                cloth=1
                if labels[j]=="SIZE":
                    size=1
                if labels[j]=="GENDER":
                    gender=1
            #    if labels[j]=="I"
            #elif df.iloc[x]['Department']=="Home":
            #else if df.iloc[x]['Department']=="Footwear":'''
        s="We are sorry for the inconvenience. "
        pro="Could you please provide "
        sp=""
        sb=""
        sg=""
        if brand==1 and product==1:
            pro=""
            s="We are sorry for the inconvenience. Please provide your contact details. Our customer support will get in touch with you shortly."
        if product==0:
            sp="the product name "
        if brand==0:
            if(sp!=""):
                sb=",the brand "
            else:
                sb="the brand "
        if gender==0 and cloth==1:
            sg=" Could you specify if it is men's wear or women's?"
        if(product==brand==0):
            pro=""
        res=s+pro+sp+sb+"?"+sg
        print(res)
        writer.writerow([res])


We are sorry for the inconvenience. Could you please provide the brand ?
We are sorry for the inconvenience. Could you please provide the brand ? Could you specify if it is men's wear or women's?
We are sorry for the inconvenience. Please provide your contact details. Our customer support will get in touch with you shortly.?
We are sorry for the inconvenience. Could you please provide the brand ?
We are sorry for the inconvenience. Could you please provide the brand ? Could you specify if it is men's wear or women's?
We are sorry for the inconvenience. Could you please provide the brand ?
We are sorry for the inconvenience. Please provide your contact details. Our customer support will get in touch with you shortly.?
We are sorry for the inconvenience. Could you please provide the brand ? Could you specify if it is men's wear or women's?
We are sorry for the inconvenience. Please provide your contact details. Our customer support will get in touch with you shortly.?
We are sorry for th

In [269]:
#Formulating the final results-review,entities recognised, Department to which the query was routed to, Automated response
df=pd.read_csv(path+"named_entities.csv", delimiter = ',')
df1=pd.read_csv(path+"response.csv", delimiter=',')
docs_new1=df["Original_review"]
docs_new2=df["Entities"]
docs_new3=df["Department"]
docs_new4=df["Label"]
docs_new5=df1["Response"]

docs_new=pd.concat([docs_new1,docs_new2,docs_new3,docs_new4,docs_new5],axis=1)
docs_new = docs_new.dropna()
with open(path+"result.csv", 'w') as writeFile:
    writer = csv.writer(writeFile)
    header=["Review","Entities","Label","Department", "Response"]
    writer.writerow(header)
    for review, entity,department,label,response in zip(docs_new["Original_review"],docs_new["Entities"],docs_new["Label"], docs_new["Department"],docs_new["Response"]):
        named_entities=[]
        #print('%r => %s' % (doc,Departments[category[0]]))
        named_entities=[review,entity,department,label,response]
        writer = csv.writer(writeFile)
        writer.writerow(named_entities)

In [286]:
df=pd.read_csv(path+"result.csv")
#Retrieve 4th row for clothing
l=[df.iloc[4].values,df.iloc[13].values,df.iloc[34].values]


In [287]:
def printmd(string,color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [288]:
for i in range(len(l)):
    printmd("**Review:**",color="teal")
    print(l[i][0],end="")
    printmd("**Entities:**",color="teal")
    print(l[i][1])
    printmd("**NER tag:**",color="teal")
    print(l[i][2])
    printmd("**Department:**",color="teal")
    print(l[i][3],end="")
    printmd("**Response:**",color="teal")
    print(l[i][4])


<span style='color:teal'>**Review:**</span>

i wore this shirt just once. it's almost transparent. not worth it even if you get it for free.


<span style='color:teal'>**Entities:**</span>

shirt


<span style='color:teal'>**NER tag:**</span>

...

PRODUCT,INFO,PRODUCT


<span style='color:teal'>**Department:**</span>

Home

<span style='color:teal'>**Response:**</span>

We are sorry for the inconvenience. Could you please provide the brand ?
